In [2]:
# Notes for me: Create same model but twist it for songs as well so I can pair both models together in utility

import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import MaxAbsScaler, Normalizer
from sklearn.pipeline import make_pipeline


# Loading clean sparse df I made 
master_df = pd.read_csv('spotify-sparse5.csv')

In [9]:
# Creating a df that is easier to view in Data Viewer 
# So i can see what I am doing without having to take up a ton of mem or do a alot of print() coding
view_mdf = master_df.iloc[:10, :100]

# Extracting a list of artist names, I will need this later on after sparse has gone through the pipeline.
artist_list = master_df['artistname'].values.tolist()

# Reindexed
data_reindex_df = master_df.set_index('artistname')

# New view
view_ridf = data_reindex_df.iloc[:10, :100]

In [ ]:
# Creating the model and training it

# WARNING WARNING WARNING WARNING WARNING
# WARNING WARNING WARNING WARNING WARNING
# WARNING WARNING WARNING WARNING WARNING
# This cell is required if you don't have a trained model, if you do, please skip this cell.
# If you are electing to ignore this warning, note that this model
# took 40 mins to train, using 8 CPU cores clocking at 4.2Ghz and 16Gb RAM.

scaler = MaxAbsScaler()
nmf = NMF(n_components=data_reindex_df.shape[1])
norm = Normalizer()
pipeline = make_pipeline(scaler, nmf, norm)

scal_nmf_norm = pipeline.fit_transform(data_reindex_df)

In [7]:
# Conversion to a df and appending the artist list as the index so I can queue the results
scal_nmf_norm_df = pd.DataFrame(scal_nmf_norm, index=artist_list)

In [ ]:
# This is for exporting the model
scal_nmf_norm_df.to_csv('sparse5-trained.csv')

In [3]:
# This cell is for importing the already trained model
scal_nmf_norm_df = pd.read_csv('sparse5-trained.csv', index_col='Unnamed: 0')


In [6]:
# Making it more fool-proof by auto formatting the input, selecting artist.
selecting_artist = str('future').title()

# Selecting the observation
artist = scal_nmf_norm_df.loc[selecting_artist]

# Scaler-product the observation to the df to get the simularity scores
recomendation = scal_nmf_norm_df.dot(artist)

# Display the results
print(recomendation.nlargest(11))

Future           1.000000
Migos            0.910087
Young Thug       0.823167
Rokia Traoré     0.774155
Vince Staples    0.722169
George Maple     0.710384
Bobbie Gentry    0.710303
Rocko            0.695946
Chief Keef       0.680778
Death Grips      0.671577
Rich Gang        0.667272
dtype: float64
